In [0]:
library(tidyverse)
library(haven)
install.packages("MatchIt"); library(MatchIt)
install.packages("WeightIt"); library(WeightIt)
install.packages("cobalt");library(cobalt)
install.packages("gbm"); library(gbm)
install.packages("CBPS"); library(CBPS)
install.packages("causalsens"); library(causalsens)

This scripts uses the famous Lalonde dataset. So, let's get it.

In [0]:
cps1_data <- read_dta("https://users.nber.org/~rdehejia/data/cps_controls.dta")
cps3_data <- read_dta("https://users.nber.org/~rdehejia/data/cps_controls3.dta")
nswdw_data <- read_dta("https://users.nber.org/~rdehejia/data/nsw_dw.dta")

In [0]:
# Extract the treatment group from NSW Data and treat it as a treatment group in CPS1.
cps1_nsw_data <- nswdw_data %>% 
    filter(treat==1) %>% 
    rbind(cps1_data)

# Extract the treatment group from NSW Data and treat it as a treatment group in CPS3.
cps3_nsw_data <- nswdw_data %>% 
    filter(treat==1) %>%
    rbind(cps3_data)

Define the formulas previously.  
In this script, we don't include "re75" as a covariate because it will be used to assess the unconfoundedness later.

In [0]:
# Formula of outcome model
frml_out <- as.formula(re78 ~ treat + age + education + black + hispanic + married + nodegree + re74 + I(re74^2))

# Formula of treatment model
frml_treat <- as.formula(treat ~ age + education + black + hispanic + married + nodegree + re74 + I(re74^2))

First, we create matching models via the "matchit()" function.


In [0]:
# nearest neighbor matching
m_out_nn_1 <- matchit(frml_treat, cps1_nsw_data, method="nearest", ratio=1, replace=TRUE, discard="control")
m_out_nn_4 <- matchit(frml_treat, cps1_nsw_data, method="nearest", ratio=4, replace=TRUE, discard="control")

# subclassification
m_out_subclass <- matchit(frml_treat, cps1_nsw_data, method="subclass", subclass=10, discard="control")

# caliper matching
m_out_caliper <- matchit(frml_treat, cps1_nsw_data, method="nearest", replace=TRUE, caliper=.01, ratio=1, discard="control")

Next, we create weighting models via the "weightit()" function.

In [0]:
# models
w_out_ps <- weightit(frml_treat, cps1_nsw_data, method="ps", estimand="ATT")
w_out_gbm <- weightit(frml_treat, cps1_nsw_data, method="gbm", stop.method="es.mean", estimand="ATT", bag.fraction=0.7, cv.folds=5)
w_out_cbps <- weightit(frml_treat, cps1_nsw_data, method="cbps", estimand="ATT")

Some extreme weights were generated.  
So, let's check the summary of weighted models, and next, trim the extreme weights via the "trim()" function.

In [0]:
print(summary(w_out_ps))
print(summary(w_out_gbm))
print(summary(w_out_cbps))

In [0]:
# Trimming and re-check the summary.
w_out_ps_trm <- w_out_ps %>% trim(at=5, lower=TRUE)
w_out_gbm_trm <- w_out_gbm %>% trim(at=2, lower=TRUE)
w_out_cbps_trm <- w_out_cbps %>% trim(at=5, lower=TRUE)
summary(w_out_ps_trm)
summary(w_out_gbm_trm)
summary(w_out_cbps_trm)

Check the covariate balance by love.plot() in "cobalt" package.

In [0]:
# matching models
love.plot(frml_treat, data=cps1_nsw_data,
          weights=data.frame(Nearest_Neighbor_1=get.w(m_out_nn_1),
                             Nearest_Neighbor_4=get.w(m_out_nn_4),
                             Subclassification=get.w(m_out_subclass),
                             Caliper=get.w(m_out_caliper)),
          method=c("matching", "matching", "matching", "matching"),
          binary="std", s.d.denom="treated", grid=FALSE, threshold=.1, limits=c(0,1))

In [0]:
# Weighting models
love.plot(frml_treat, data=cps1_nsw_data,
          weights=data.frame(PS=get.w(w_out_ps_trm),
                             GBM=get.w(w_out_gbm_trm),
                             CBPS=get.w(w_out_cbps_trm)),
          method=c("weighting", "weighting", "weighting"),
          binary="std", s.d.denom="treated", grid=FALSE, threshold=.1, limits=c(0,1))

In [0]:
# check the covariate balance per covariate
bal.plot(m_out_nn_1, var.name="distance", which="both", type="histogram", mirror=TRUE)

Get the estimated ATT via the linear model.

In [0]:
# matching methods
matched_data <- list(match.data(m_out_nn_1),
                     match.data(m_out_nn_4),
                     match.data(m_out_subclass),
                     match.data(m_out_caliper))
model_label1 <- c("nearest neighbor ratio 1", "nearest neighbor ratio 4", "subclassification", "caliper")

for(i in 1:length(matched_data)){
  print(model_label1[[i]])
  print(lm(re78 ~ treat, data=matched_data[[i]]) %>% broom::tidy())
}

In [0]:
# weighting methods
weighted_data <- list(w_out_ps_trm, w_out_gbm_trm, w_out_cbps_trm)
model_label2 <- c("PS", "GBM", "CBPS")

for(i in 1:length(weighted_data)){
  print(model_label2[[i]])
  print(lm(re78 ~ treat, data=cps1_nsw_data, weights=weighted_data[[i]]$weights) %>% broom::tidy())
}

Doubly Robust Estimator.

In [0]:
# weighting methods
weighted_data <- list(w_out_ps_trm, w_out_gbm_trm, w_out_cbps_trm)
model_label2 <- c("PS", "GBM", "CBPS")

for(i in 1:length(weighted_data)){
  print(model_label2[[i]])
  print(lm(frml_out, data=cps1_nsw_data, weights=weighted_data[[i]]$weights) %>% broom::tidy())
}

Sensitivity analysis.

In [0]:
model_out <- lm(frml_out, data=cps1_nsw_data)
model_treat <- glm(frml_treat, data=cps1_nsw_data, family=binomial())
alpha <- seq(-4500, 4500, by = 250)

ll_sens <- causalsens(model_out, model_treat, ~ age + education, data=cps1_nsw_data, confound=one.sided.att, alpha=alpha)

plot(ll_sens, type="raw", bty="n")
plot(ll_sens, type="r.squared", bty="n")

Unconfoundedness assessment.  
We regress the lag variable "re75" by the covariates in order to assess the unconfoundedness.  
If the result is null or ignorably small, it can be regarded as one of the clue of the unconfoundedness.

In [0]:
# matching methods
matched_data <- list(match.data(m_out_nn_1),
                     match.data(m_out_nn_4),
                     match.data(m_out_subclass),
                     match.data(m_out_caliper))
model_label1 <- c("nearest neighbor ratio 1", "nearest neighbor ratio 4", "subclassification", "caliper")

for(i in 1:length(matched_data)){
  print(model_label1[[i]])
  print(lm(re75 ~ treat, data=matched_data[[i]]) %>% broom::tidy())
}

In [0]:
# weighting methods
weighted_data <- list(w_out_ps_trm, w_out_gbm_trm, w_out_cbps_trm)
model_label2 <- c("PS", "GBM", "CBPS")

for(i in 1:length(weighted_data)){
  print(model_label2[[i]])
  print(lm(re75 ~ treat, data=cps1_nsw_data, weights=weighted_data[[i]]$weights) %>% broom::tidy())
}

The outcomes of the treatment / control groups are statistically different.  
So, there may be unadjusted confounding factors in our models.